<a href="https://colab.research.google.com/github/Tanayshah29/Mnist_Fashion_CNN/blob/main/Mnist_Fashion_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create a CNN and OPtimize it using Keras Tuner

In [25]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [26]:
print(tf.__version__)

2.3.0


In [27]:
fashion_mnist = keras.datasets.fashion_mnist

In [28]:
(train_images, train_labels), (test_images, test_labels)=fashion_mnist.load_data()

In [29]:
train_images = train_images/255.0
test_images = test_images/255.0

In [30]:
train_images[0].shape

(28, 28)

In [31]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [35]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [36]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [37]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name="MNIST Fashion")

In [38]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4081 - accuracy: 0.8509 - val_loss: 0.3145 - val_accuracy: 0.8808
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2695 - accuracy: 0.8998 - val_loss: 0.2948 - val_accuracy: 0.8887
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2160 - accuracy: 0.9204 - val_loss: 0.2636 - val_accuracy: 0.9047


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.5031 - accuracy: 0.8286 - val_loss: 0.4051 - val_accuracy: 0.8450
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3771 - accuracy: 0.8620 - val_loss: 0.3520 - val_accuracy: 0.8672
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3294 - accuracy: 0.8767 - val_loss: 0.3539 - val_accuracy: 0.8678


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3881 - accuracy: 0.8596 - val_loss: 0.2872 - val_accuracy: 0.8952
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2475 - accuracy: 0.9084 - val_loss: 0.2600 - val_accuracy: 0.9042
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1879 - accuracy: 0.9305 - val_loss: 0.2357 - val_accuracy: 0.9140


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4137 - accuracy: 0.8511 - val_loss: 0.3155 - val_accuracy: 0.8852
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2639 - accuracy: 0.9024 - val_loss: 0.2738 - val_accuracy: 0.8988
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2070 - accuracy: 0.9223 - val_loss: 0.2747 - val_accuracy: 0.9010


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4051 - accuracy: 0.8531 - val_loss: 0.3040 - val_accuracy: 0.8877
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2657 - accuracy: 0.9007 - val_loss: 0.3087 - val_accuracy: 0.8802
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2077 - accuracy: 0.9225 - val_loss: 0.2525 - val_accuracy: 0.9095


INFO:tensorflow:Oracle triggered exit


In [39]:
model = tuner_search.get_best_models(num_models=1)[0]

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        96048     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2973824   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 3,071,962
Trainable params: 3,071,962
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1395 - accuracy: 0.9472 - val_loss: 0.2583 - val_accuracy: 0.9150
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0989 - accuracy: 0.9628 - val_loss: 0.2868 - val_accuracy: 0.9118
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0702 - accuracy: 0.9730 - val_loss: 0.3273 - val_accuracy: 0.9110
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0515 - accuracy: 0.9809 - val_loss: 0.3467 - val_accuracy: 0.9135
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0369 - accuracy: 0.9866 - val_loss: 0.4404 - val_accuracy: 0.9108
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0312 - accuracy: 0.9894 - val_loss: 0.4649 - val_accuracy: 0.9068
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0274 - accuracy: 0.9904 - val_loss: 0.4938 - val_accuracy